# GRU Classification with SUBJ Dataset
<hr>

We will build a text classification model using GRU model on the SUBJ Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


sentence  label  split
0     smart and alert , thirteen conversations about...      0  train
1     color , musical bounce and warm seas lapping o...      0  train
2     it is not a mass market entertainment but an u...      0  train
3     a light hearted french film about the spiritua...      0  train
4     my wife is an actress has its moments in looki...      0  train
...                                                 ...    ...    ...
9995  in the end , they discover that balance in lif...      1  train
9996  a counterfeit 1000 tomin bank note is passed i...      1  train
9997  enter the beautiful and mysterious secret agen...      1  train
9998  after listening to a missionary from china spe...      1  train
9999  looking for a short cut to fame , glass concoc...      1  train

[10000 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          5000   5000
1          5000   5000

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 487,009
Trainable params: 487,009
Non-trainable params: 0
_________________________________________________________________


In [13]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [14]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
282/282 [==============================] - 84s 257ms/step - loss: 0.4351 - accuracy: 0.7757 - val_loss: 0.2216 - val_accuracy: 0.9150
Epoch 2/15
282/282 [==============================] - 70s 248ms/step - loss: 0.0813 - accuracy: 0.9736 - val_loss: 0.2411 - val_accuracy: 0.9010
Epoch 3/15
282/282 [==============================] - 74s 264ms/step - loss: 0.0211 - accuracy: 0.9952 - val_loss: 0.3469 - val_accuracy: 0.9070
Epoch 4/15
282/282 [==============================] - 71s 253ms/step - loss: 0.0108 - accuracy: 0.9977 - val_loss: 0.4108 - val_accuracy: 0.8900
Epoch 5/15
282/282 [==============================] - 75s 267ms/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.4150 - val_accuracy: 0.9140
Epoch 6/15
282/282 [==============================] - 79s 279ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.4509 - val_accuracy: 0.9040
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 91.50000214576721


Epoch 2/15
282/282 [==============================] - 139s 491ms/step - loss: 0.0793 - accuracy: 0.9754 - val_loss: 0.2832 - val_accuracy: 0.9080
Epoch 3/15
282/282 [==============================] - 134s 475ms/step - loss: 0.0204 - accuracy: 0.9939 - val_loss: 0.3074 - val_accuracy: 0.9020
Epoch 4/15
282/282 [==============================] - 132s 467ms/step - loss: 0.0067 - accuracy: 0.9987 - val_loss: 0.3414 - val_accuracy: 0.9070
Epoch 5/15
282/282 [==============================] - 130s 463ms/step - loss: 0.0038 - accuracy: 0.9996 - val_loss: 0.4280 - val_accuracy: 0.8930
Epoch 6/15
282/282 [==============================] - 130s 462ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 0.4531 - val_accuracy: 0.8860
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 92.5000011920929
Training 10: 
Epoch 1/15
282/282 [==============================] - 113s 365ms/step - loss: 0.4532 - accuracy: 0.7945 - val_loss: 0.2112 - val_accuracy: 0

## Summary

In [15]:
record

acc1       acc2       acc3       acc4       acc5       acc6  \
0  91.500002  91.600001  92.199999  90.600002  92.900002  90.399998   

        acc7  acc8       acc9      acc10        AVG  
0  91.000003  93.5  92.500001  92.400002  91.860001

In [16]:
report = record
report = report.to_excel('GRU_SUBJ_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [18]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [19]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 523,361
Trainable params: 223,361
Non-trainable params: 300,000
____________________________________________

## Train and Test the Model

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [21]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
282/282 [==============================] - 126s 366ms/step - loss: 0.3842 - accuracy: 0.8334 - val_loss: 0.2478 - val_accuracy: 0.9100
Epoch 2/40
282/282 [==============================] - 90s 321ms/step - loss: 0.2153 - accuracy: 0.9181 - val_loss: 0.2301 - val_accuracy: 0.9080
Epoch 3/40
282/282 [==============================] - 102s 360ms/step - loss: 0.1838 - accuracy: 0.9319 - val_loss: 0.2335 - val_accuracy: 0.8990
Epoch 4/40
282/282 [==============================] - 102s 360ms/step - loss: 0.1442 - accuracy: 0.9464 - val_loss: 0.2594 - val_accuracy: 0.8970
Epoch 5/40
282/282 [==============================] - 95s 336ms/step - loss: 0.1208 - accuracy: 0.9566 - val_loss: 0.2375 - val_accuracy: 0.9090
Epoch 6/40
282/282 [==============================] - 88s 313ms/step - loss: 0.0968 - accuracy: 0.9622 - val_loss: 0.3644 - val_accuracy: 0.8950
Epoch 7/40
282/282 [==============================] - 88s 314ms/step - loss: 0.0683 - accuracy: 0.9755 - val_loss:

282/282 [==============================] - 126s 445ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.6163 - val_accuracy: 0.9210
Epoch 20/40
282/282 [==============================] - 131s 465ms/step - loss: 0.0080 - accuracy: 0.9970 - val_loss: 0.6929 - val_accuracy: 0.9210
Epoch 21/40
282/282 [==============================] - 126s 446ms/step - loss: 0.0112 - accuracy: 0.9967 - val_loss: 0.6916 - val_accuracy: 0.9160
Epoch 22/40
282/282 [==============================] - 131s 466ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 0.7124 - val_accuracy: 0.9160
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Test Accuracy: 92.79999732971191
Training 5: 
Epoch 1/40
282/282 [==============================] - 121s 361ms/step - loss: 0.3874 - accuracy: 0.8293 - val_loss: 0.2814 - val_accuracy: 0.8900
Epoch 2/40
282/282 [==============================] - 99s 352ms/step - loss: 0.2235 - accuracy: 0.9114 - val_loss: 0.2119 - val_accuracy: 0.9120
Epo

Epoch 15/40
282/282 [==============================] - 161s 571ms/step - loss: 0.0104 - accuracy: 0.9986 - val_loss: 0.6247 - val_accuracy: 0.8950
Epoch 16/40
282/282 [==============================] - 159s 563ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.5943 - val_accuracy: 0.9130
Epoch 17/40
282/282 [==============================] - 158s 559ms/step - loss: 0.0075 - accuracy: 0.9976 - val_loss: 0.6179 - val_accuracy: 0.9100
Epoch 18/40
282/282 [==============================] - 162s 576ms/step - loss: 0.0119 - accuracy: 0.9947 - val_loss: 0.6936 - val_accuracy: 0.9060
Epoch 19/40
282/282 [==============================] - 168s 597ms/step - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.6687 - val_accuracy: 0.9210
Epoch 20/40
282/282 [==============================] - 170s 602ms/step - loss: 0.0166 - accuracy: 0.9946 - val_loss: 0.5542 - val_accuracy: 0.9200
Epoch 21/40
282/282 [==============================] - 173s 612ms/step - loss: 0.0095 - accuracy: 0.9963 - val_loss: 0

Epoch 21/40
282/282 [==============================] - 142s 505ms/step - loss: 0.0055 - accuracy: 0.9985 - val_loss: 0.5231 - val_accuracy: 0.9180
Epoch 22/40
282/282 [==============================] - 142s 504ms/step - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.5399 - val_accuracy: 0.9210
Epoch 23/40
282/282 [==============================] - 134s 476ms/step - loss: 0.0114 - accuracy: 0.9958 - val_loss: 0.5234 - val_accuracy: 0.9240
Epoch 24/40
282/282 [==============================] - 135s 480ms/step - loss: 0.0056 - accuracy: 0.9971 - val_loss: 0.6077 - val_accuracy: 0.9190
Epoch 25/40
282/282 [==============================] - 136s 483ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.5933 - val_accuracy: 0.9300
Epoch 26/40
282/282 [==============================] - 136s 483ms/step - loss: 2.1616e-04 - accuracy: 1.0000 - val_loss: 0.6158 - val_accuracy: 0.9240
Epoch 27/40
282/282 [==============================] - 137s 485ms/step - loss: 1.2342e-04 - accuracy: 1.0000 - val

## Summary

In [22]:
record2

acc1       acc2       acc3       acc4       acc5       acc6  \
0  91.000003  93.300003  90.899998  92.799997  93.599999  92.400002   

        acc7       acc8  acc9      acc10    AVG  
0  91.299999  93.199998  91.7  92.500001  92.27

In [23]:
report = record2
report = report.to_excel('GRU_SUBJ_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 128)               74496     
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 523,361
Trainable params: 523,361
Non-trainable params: 0
_______________________________________________

## Train and Test the Model

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [27]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 
Epoch 1/40
282/282 [==============================] - 307s 1s/step - loss: 0.3793 - accuracy: 0.8234 - val_loss: 0.1610 - val_accuracy: 0.9320
Epoch 2/40
282/282 [==============================] - 255s 906ms/step - loss: 0.0877 - accuracy: 0.9714 - val_loss: 0.1724 - val_accuracy: 0.9300
Epoch 3/40
282/282 [==============================] - 236s 838ms/step - loss: 0.0234 - accuracy: 0.9938 - val_loss: 0.2285 - val_accuracy: 0.9230
Epoch 4/40
282/282 [==============================] - 239s 849ms/step - loss: 0.0102 - accuracy: 0.9965 - val_loss: 0.4044 - val_accuracy: 0.9240
Epoch 5/40
282/282 [==============================] - 240s 851ms/step - loss: 0.0047 - accuracy: 0.9982 - val_loss: 0.4085 - val_accuracy: 0.9260
Epoch 6/40
282/282 [==============================] - 244s 864ms/step - loss: 5.8330e-04 - accuracy: 0.9998 - val_loss: 0.3744 - val_accuracy: 0.9240
Epoch 7/40
282/282 [==============================] - 251s 888ms/step - loss: 2.8277e-04 - accuracy: 1.0000 - 

282/282 [==============================] - 425s 2s/step - loss: 0.0035 - accuracy: 0.9985 - val_loss: 0.6262 - val_accuracy: 0.9140
Epoch 8/40
282/282 [==============================] - 427s 2s/step - loss: 6.1360e-04 - accuracy: 0.9998 - val_loss: 0.6428 - val_accuracy: 0.9120
Epoch 9/40
282/282 [==============================] - 427s 2s/step - loss: 2.7954e-04 - accuracy: 1.0000 - val_loss: 0.6861 - val_accuracy: 0.9130
Epoch 10/40
282/282 [==============================] - 425s 2s/step - loss: 2.4867e-05 - accuracy: 1.0000 - val_loss: 0.7131 - val_accuracy: 0.9140
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
Test Accuracy: 93.00000071525574
Training 7: 
Epoch 1/40
282/282 [==============================] - 396s 1s/step - loss: 0.3740 - accuracy: 0.8213 - val_loss: 0.2059 - val_accuracy: 0.9180
Epoch 2/40
282/282 [==============================] - 279s 988ms/step - loss: 0.0878 - accuracy: 0.9720 - val_loss: 0.2478 - val_accuracy: 0.9220
Epoch 3

Test Accuracy: 93.69999766349792

        acc1       acc2       acc3       acc4       acc5       acc6  \
0  93.199998  91.100001  93.099999  92.000002  93.300003  93.000001   

        acc7       acc8       acc9      acc10        AVG  
0  92.400002  93.300003  91.799998  93.699998  92.690001  



## Summary

In [28]:
record3

acc1       acc2       acc3       acc4       acc5       acc6  \
0  93.199998  91.100001  93.099999  92.000002  93.300003  93.000001   

        acc7       acc8       acc9      acc10        AVG  
0  92.400002  93.300003  91.799998  93.699998  92.690001

In [29]:
report = record3
report = report.to_excel('GRU_SUBJ_v2_3.xlsx', sheet_name='dynamic')